In [26]:
# ***********************************
# Filename: 분석보고서_통계_산출.py 
# Author: Haeun Jeong
# 오라클에 적재된 데이터를 기준으로 통계치 산출
# 
# **********************************


# **********************************
# Module Importation
# **********************************

import cx_Oracle
import os
import csv
from datetime import date
import glob
import pandas as pd
from tqdm import tqdm_notebook
import numpy as np


# **********************************
# Setting OS ENV
# **********************************

os.putenv('NLS_LANG', 'KOREAN_KOREA.KO16MSWIN949')


# **********************************
# Global Constant
# **********************************

UID = 'sibum'
PW = 'imsi00.!'
HOST_NAME = '10.46.107.124:22751/SIBUMDB'


# **********************************
# Global Variable
# **********************************


today = "20200810" # 통계를 내려는 날짜

has_error = False

table_name = "HJ_REPORT_INFO"

df = ""
df_score = ""
# **********************************
# Define Class
# **********************************

class Oracle:
    uid = UID
    pw = PW
    host_name = HOST_NAME
    connection = None
    
    def __init__(self): # 생성 시 DB 연결 및 
        self.connect_db()

        
    def __del__(self): # 소멸 시 DB 연결 해제
        return self.close_db()

    
    def connect_db(self):
        self.connection = cx_Oracle.connect(self.uid, self.pw, self.host_name)
        self.cursor = self.connection.cursor()
        return self.cursor
    
    
    def reconnect_db(self):
        self.close_db()
        return self.connect_db
    
    
    def close_db(self):
        if self.connection != None:
            self.connection.close()
            return 0
        else:
            return 1
    
    def execute_query(self, query, check_error = False):
        try:
            self.cursor.execute(query)
#             print("성공")
            return False
        except Exception as e:
            print(query)
            print(f"에러가 발생했습니다. {str(e)}")
            if check_error:
                return True
    
    
    def print_query_result(self):
        self.cursor.rowfactory = make_dict_factory(self.cursor)
        self.query_results = self.cursor.fetchall()
        for query in self.query_results:
            print(query)


# **********************************
# Entry Point
# **********************************

if __name__ == "__main__":                 
        
        print("-- 프로그램 시작 --")
        oracle = Oracle()
        
        # 등급, seg 별 enc 개수 출력
        query = f"""SELECT r_fin_gdr AS 등급,
                    CASE WHEN model_grp = '단기체납' THEN 'SEG1'
                    WHEN model_grp = '장기체납' THEN 'SEG2'
                    WHEN model_grp = '결손처분' THEN 'SEG3'
                    END AS SEG,
                    COUNT(DISTINCT tpr_no_enc)
                    FROM hj_report_info
                    WHERE model_grp != '미산출대상' AND today = {today}
                    GROUP BY model_grp, r_fin_gdr
                    ORDER BY model_grp DESC, r_fin_gdr ASC"""
        df = pd.read_sql(query, con=oracle.connection)
        df_pivot = pd.pivot_table(df, values='COUNT(DISTINCTTPR_NO_ENC)', index = ['등급'], columns = ['SEG'], aggfunc=np.sum)
        df_pivot = df_pivot.replace(np.nan, '', regex = True)
        display(df_pivot)
        # score, seg 별 enc 개수 출력
        score_query = f"""
                     SELECT  SEG, scores, COUNT(distinct cnt)
                        FROM (
                          SELECT
                               CASE WHEN model_grp = '단기체납' THEN 'SEG1'
                                    WHEN model_grp = '장기체납' THEN 'SEG2'
                                    WHEN model_grp = '결손처분' THEN 'SEG3'
                               END AS SEG,
                           CASE WHEN FLOOR(r_fin_score) BETWEEN 950 AND 1000 THEN '950'
                                    WHEN FLOOR(r_fin_score) BETWEEN 900 AND 949 THEN '900'
                                    WHEN FLOOR(r_fin_score) BETWEEN 850 AND 899 THEN '850'
                                    WHEN FLOOR(r_fin_score) BETWEEN 800 AND 849 THEN '800'
                                    WHEN FLOOR(r_fin_score) BETWEEN 750 AND 799 THEN '750'
                                    WHEN FLOOR(r_fin_score) BETWEEN 700 AND 749 THEN '700'
                                    WHEN FLOOR(r_fin_score) BETWEEN 650 AND 699 THEN '650'
                                    WHEN FLOOR(r_fin_score) BETWEEN 600 AND 649 THEN '600'
                                    WHEN FLOOR(r_fin_score) BETWEEN 550 AND 599 THEN '550'
                                    WHEN FLOOR(r_fin_score) BETWEEN 500 AND 549 THEN '500'
                                    WHEN FLOOR(r_fin_score) BETWEEN 450 AND 499 THEN '450'
                                    WHEN FLOOR(r_fin_score) BETWEEN 400 AND 449 THEN '400'
                                    WHEN FLOOR(r_fin_score) BETWEEN 350 AND 399 THEN '350'
                                    WHEN FLOOR(r_fin_score) BETWEEN 300 AND 349 THEN '300'
                                    WHEN FLOOR(r_fin_score) BETWEEN 250 AND 299 THEN '250'
                                    WHEN FLOOR(r_fin_score) BETWEEN 200 AND 249 THEN '200'
                                    WHEN FLOOR(r_fin_score) BETWEEN 150 AND 199 THEN '150'
                                    WHEN FLOOR(r_fin_score) BETWEEN 100 AND 149  THEN '100'
                                    WHEN FLOOR(r_fin_score) BETWEEN 50 AND 99  THEN '50'
                                    WHEN FLOOR(r_fin_score) BETWEEN 0 AND 49 THEN '0'
                               END AS scores,
                               tpr_no_enc as cnt
                               FROM hj_report_info
                               WHERE today = {today}
                        ) hj_report_info
                        WHERE SEG != '미산출대상'
                        GROUP BY  SEG, scores
                        ORDER BY SEG ASC, scores desc"""
        df_score = pd.read_sql(score_query, con=oracle.connection)
        df_score_pivot = pd.pivot_table(df_score, values='COUNT(DISTINCTCNT)', index = ['SCORES'], columns = ['SEG'], aggfunc=np.sum)
        df_score_pivot = df_score_pivot.replace(np.nan, '', regex = True)
        df_score_pivot = df_score_pivot.sort_index(ascending = False)
        display(df_score_pivot)
        if os.path.exists(f"./{today}_분석보고서_통계_산출.xlsx"): # 해당 날짜의 엑셀 파일이 기존에 있으면 해당 엑셀파일을 지운다.
            os.remove(f"{today}_분석보고서_통계_산출.xlsx")
        
        with pd.ExcelWriter(f"{today}_분석보고서_통계_산출.xlsx")as writer:
            df_pivot.to_excel(writer, sheet_name = "seg_등급_통계")
            df_score_pivot.to_excel(writer, sheet_name = "seg_score_통계")
        print("-- 프로그램 종료 --")


-- 프로그램 시작 --


SEG,SEG1,SEG2,SEG3
등급,,,
1,7246.0,,
2,147051.0,691,
3,148741.0,116236,1343
4,15523.0,230726,10400
5,8.0,185842,19625


SEG,SEG1,SEG2,SEG3
SCORES,,,
950,18769,,
900,90101,80,
850,86024,4761,
800,50709,24822,
750,33254,37537,526
700,21903,44244,698
650,13225,46823,1346
600,4007,59480,2431
550,540,92700,4446


-- 프로그램 종료 --
